## Ordinary Differential Equations (ODEs)

In [1]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

<IPython.core.display.Javascript object>

### Notations

Before we start, let's define some useful notations. The class of problems we try to solve using Simwork is a subet of the ODEs family called Initial Value Problems (IVPs). They have the following form:

\begin{equation}
\dot y = f(y,t), \quad y(t_0) = y_0,
\end{equation}

where $y, y_0 \in \mathbb R^n$. Here $n$ corresponds to the **dimension** of the problem or, in other words, the number of unknowns. In our world, $n$ usually corresponds to the number of entities (or `BioReac` in Simwork).
$n$ can range from 1 to 500+ for very complex models. As a rule of thumb, the higher the $n$, the more difficult it is to solve the problem, but we'll come back to that later.

### Numerical Methods

Among the vast collection of methods used to solve IVPs problems, we shall focus here on the implicit Backward Differentiation Formula (BDF) family because it's one of the methods (and actually almost the only one) we use in practice. It has proved to be very robust and efficient when solving large and _stiff_ problems.

### The fixed-order BDF scheme

To solve (1), the following discretization is proposed, where $t_n$ is the $n$-th discrete time point:

\begin{align}
y(t_n) &\simeq y^n, \\
f(y(t_n), t_n) &\simeq \beta f(y^n, t_n), \\
\dot y(t_n) &\simeq \frac{1}{h} \sum_{i=0}^{p} \alpha_i y(t^{n-i}),
\end{align}

where $p$ is the **order** of the method. The $p$-th order BDF scheme is usually referred to as BDF-p.
In practice, only the methods from BDF-1 (a.k.a _implicit Euler_) to BDF-6 (BDF-6 is so unstable that the library we're using stops at BDF-5) are used, the higher order methods being too unstable.
The higher the order, the more accurate the method is.  
Indeed, it can be shown that the truncation error for the $p$-order method is $\mathcal O (h^p)$, meaning bigger time steps can be taken to reach a given accuracy.  
Unfortunately, higher order also means more instability such that starting for $p = 3, 4, 5$ the methods are usually unusable in practice _as is_. That's where adaptive order BDF come into action.

### Adaptive order BDF

In practice we use all BDFs (BDF1 -> BDF5) at once ! This hybrid scheme is called adaptive order because it automatically adapts the order of the method during the time integration based on the current stability and the desired accuracy.

### Material used to write this notebook

Süli, Mayers, An Introduction to Numerical Analysis, Cambridge University Press, 2003.